# This examples shows how to interact with the ONC API and additional featrues from `strawb`
The methods 

In [ ]:
import pandas
import plotly.express as px
import numpy as np

import strawb

In [ ]:
onc_downloader = strawb.ONCDownloader(showInfo=False, timeout=60)

# select a device: e.g. 'ONCMJB016' the STRAWb MiniJB; or a module: 'TUMLIDAR001', 'TUMMUONTRACKER001',..
filters = {'deviceCode': 'ONCMJB016',
           'dateFrom': '2021-10-11T00:00:00.000Z',
           'dateTo': '2021-10-12T00:00:00.000Z',
           # 'extension': 'hdf5'
          }

# More `filters` parameter are: 
# 'dataProductCode': 'SMRD'  # use onc_downloader.getDataProducts(...) (see below) to check for posible entries 
# 'extension': 'hdf5'  # use onc_downloader.getDataProducts(...) (see below) to check for posible entries

# 1. Get available files from the ONC server

## 1.1. Basic method from the ONC API (not recomended)
the returned dict has a list with filenames under 'files'

In [ ]:
onc_downloader.getListByDevice(filters=filters, allPages=True)

## 1.2. STRAWb method (recomended): 
### 1.2.1 Get the files as a Pandas DataFrame + a structured 'outPath'
The columns are infromations from the ONC DB. The column 'synced' shows if the file is available localy.

In [ ]:
df_result = onc_downloader.get_files_structured(dev_codes=[filters['deviceCode']],
                                                date_from=filters['dateFrom'],
                                                date_to=filters['dateTo'],
#                                                 extensions=['txt', 'hdf5', 'hld']
                                               )

df_result  # show the pandas DataFrame

### 1.2.2 Mask the resulting DataFrame this has more features as provided by the filters

In [ ]:
mask = df_result['fileSize']<10e3
df_result[mask]  # show the filterd files

# 2. Download files 

## 2.1 Basic method from the ONC API (not recomended -> #)

In [ ]:
# onc_downloader.getDirectFiles(filters_or_result=filters)

## 2.2 STRAWb method (recomended)
use the mask and DataFrame from 1.2.1 and 1.2.2

In [ ]:
onc_downloader.getDirectFiles(filters_or_result=df_result[mask])

## 2.3 STRAWb method (recomended, too)

In [ ]:
# as standalone
onc_downloader.download_structured(dev_codes=[filters['deviceCode']],
                                   date_from=filters['dateFrom'],
                                   date_to=filters['dateTo'],
                                   max_file_size=10e3,
#                                  extensions=['txt', 'hdf5', 'hld']
                                  )

In [ ]:
# based on the DataFrame
onc_downloader.download_structured(pd_result=df_result[mask])

### Access the info about the download
works with all options from 2.

In [ ]:
onc_downloader.result

# 3. Explore the ONC DB
## 3.1 Get Data Products
possible dataProductCodes and dataProductName for a device

In [ ]:
# get possible dataProductCodes and dataProductName for the device
onc_downloader.getDataProducts(filters={'deviceCode': filters['deviceCode'],
                                        # 'extension': 'hdf5',  # more filter options possible
                                        # 'dataProductCode': 'LF',
                                        }
                              )

## 3.2 Get Locations

In [ ]:
result = onc_downloader.getLocations()

result

In [ ]:
# convert the result to a DataFrame
df = pandas.DataFrame(data=result)

# filter out locations which are located at 'Cascadia' Basin
mask = df.description.str.contains('Cascadia')
mask &= ~df.description.isnull()

# show the dataframe
df[mask]

In [ ]:
# Style dataframe for plotting

# add location column to group data
df['location'] = 'Others'
df.loc[mask, 'location'] = 'Cascadia Basin'

# mask invalid parameters that `fig.update_geos(fitbounds="locations")` works
mask_invalid = df.lon.abs() > 180.
mask_invalid | df.lat.abs() > 90.

# detect positions where either 'lon', 'lat' or both is Nan
# must be `mask_invalid = ` here <-> pandas...
mask_invalid = mask_invalid | df[['lon', 'lat']].isnull().any(axis=1)

In [ ]:
# Plot locations
fig = px.scatter_geo(df[~mask_invalid],
                    lat='lat',
                    lon='lon',
                    color='location',
                    hover_data=["locationName", 'locationCode'])

fig.update_geos(fitbounds="locations")

# to tune the plot
# fig.update_layout(
#     geo = dict(
#         #scope = 'north america',
#         showland = True,
#         #showlakes = True,
#         #showsubunits = True,
#         showcountries = True,
#         #showrivers=True, #rivercolor="Blue",
#         resolution = 110,  # either 110 or 50
#         #projection = dict(
#         #    type = "natural earth",
#         #    rotation_lon = -100
#         #),
#         lonaxis = dict(
#             showgrid = True,
#             gridwidth = .5,
#             #range= [ -140.0, -55.0 ],
#             dtick = 5
#         ),
#         lataxis = dict (
#             showgrid = True,
#             gridwidth = .5,
#             #range= [ 30.0, 90.0 ],
#             dtick = 5
#         )
#     ),
# )

fig.show()